In [1]:
# reverse_math_framework.py
# Python 3 code (dataclasses used). Replace with simple classes for Python 2.

from collections import defaultdict, deque
from dataclasses import dataclass, field
from typing import Dict, Set, List, Optional

@dataclass
class Subsystem:
    name: str
    rank: int  # lower rank = weaker subsystem

@dataclass
class ReverseMathDB:
    subsystems: Dict[str, Subsystem] = field(default_factory=dict)
    base_proofs: Dict[str, Set[str]] = field(default_factory=lambda: defaultdict(set))
    implications: Dict[str, Set[str]] = field(default_factory=lambda: defaultdict(set))
    theorem_meta: Dict[str, Dict] = field(default_factory=dict)

    def add_subsystem(self, name: str, rank: int):
        self.subsystems[name] = Subsystem(name=name, rank=rank)

    def add_theorem(self, name: str, meta: Optional[Dict] = None):
        if name not in self.theorem_meta:
            self.theorem_meta[name] = meta or {}

    def add_base_proof(self, subsystem: str, theorem: str):
        if subsystem not in self.subsystems:
            raise KeyError("Unknown subsystem: {}".format(subsystem))
        self.add_theorem(theorem)
        self.base_proofs[subsystem].add(theorem)

    def add_implication(self, premise: str, conclusion: str):
        self.add_theorem(premise)
        self.add_theorem(conclusion)
        self.implications[premise].add(conclusion)

    def compute_closure_per_subsystem(self):
        closure = {s: set(bases) for s, bases in self.base_proofs.items()}
        for s in self.subsystems:
            closure.setdefault(s, set())
        adj = self.implications
        for s in closure:
            queue = deque(closure[s])
            while queue:
                th = queue.popleft()
                for nxt in adj.get(th, []):
                    if nxt not in closure[s]:
                        closure[s].add(nxt)
                        queue.append(nxt)
        return closure

    def weakest_subsystem_for(self, theorem: str):
        closure = self.compute_closure_per_subsystem()
        proving = [s for s, theorems in closure.items() if theorem in theorems]
        if not proving:
            return None
        min_rank = min(self.subsystems[s].rank for s in proving)
        minimal = sorted([s for s in proving if self.subsystems[s].rank == min_rank])
        return minimal

    def explain_proof_path(self, theorem: str, subsystem: str, max_steps: int = 500):
        if subsystem not in self.subsystems:
            raise KeyError("Unknown subsystem: {}".format(subsystem))
        base = self.base_proofs.get(subsystem, set())
        if theorem in base:
            return [theorem + " (axiom/base in {})".format(subsystem)]
        parents = {}
        queue = deque()
        for b in base:
            queue.append(b)
            parents[b] = None
        visited = set(base)
        steps = 0
        while queue and steps < max_steps:
            steps += 1
            cur = queue.popleft()
            for nxt in self.implications.get(cur, []):
                if nxt not in visited:
                    parents[nxt] = cur
                    if nxt == theorem:
                        path = [nxt]
                        while parents[path[-1]] is not None:
                            path.append(parents[path[-1]])
                        path.reverse()
                        if path and path[0] in base:
                            path[0] = path[0] + " (axiom/base in {})".format(subsystem)
                        return path
                    visited.add(nxt)
                    queue.append(nxt)
        return None

    def add_example_database(self):
        # Subsystems ordered by rank (lower => weaker)
        self.add_subsystem("RCA0", rank=0)
        self.add_subsystem("WKL0", rank=1)
        self.add_subsystem("ACA0", rank=2)
        self.add_subsystem("ATR0", rank=3)
        self.add_subsystem("Pi11-CA0", rank=4)

        self.add_base_proof("RCA0", "Basic arithmetic / Sigma^0_1 comprehension instances")
        self.add_base_proof("WKL0", "Compactness of Cantor space (Weak König's Lemma)")
        self.add_base_proof("ACA0", "Arithmetical comprehension axioms")
        self.add_base_proof("ATR0", "Arithmetical transfinite recursion")
        self.add_base_proof("Pi11-CA0", "Π^1_1-comprehension")

        self.add_base_proof("WKL0", "Heine-Borel compactness for [0,1] (countable covers)")
        self.add_base_proof("ACA0", "Bolzano-Weierstrass theorem (every bounded sequence has a convergent subsequence)")
        self.add_base_proof("ATR0", "Comparability of well-orderings")
        self.add_base_proof("Pi11-CA0", "Determinacy of some projective games")

        self.add_implication("Compactness of Cantor space (Weak König's Lemma)", "Heine-Borel compactness for [0,1] (countable covers)")
        self.add_implication("Heine-Borel compactness for [0,1] (countable covers)", "Existence of suprema for bounded sequences of rationals")
        self.add_implication("Existence of suprema for bounded sequences of rationals", "Bolzano-Weierstrass theorem (every bounded sequence has a convergent subsequence)")

        self.add_base_proof("RCA0", "Ramsey's theorem for singletons (trivial)")
        self.add_base_proof("ACA0", "Some instances of Ramsey-like combinatorial principles (illustrative)")
        self.add_implication("Some instances of Ramsey-like combinatorial principles (illustrative)", "Ramsey's theorem (pairs) (illustrative)")

        self.theorem_meta.setdefault("Bolzano-Weierstrass theorem (every bounded sequence has a convergent subsequence)", {})["note"] = \
            "Illustrative: often cited as equivalent to ACA0 in many classical formulations; user should verify in literature."
        self.theorem_meta.setdefault("Heine-Borel compactness for [0,1] (countable covers)", {})["note"] = \
            "Illustrative: often connected with WKL0 in countable-cover formulations."

# Demonstration
if __name__ == "__main__":
    db = ReverseMathDB()
    db.add_example_database()
    queries = [
        "Bolzano-Weierstrass theorem (every bounded sequence has a convergent subsequence)",
        "Heine-Borel compactness for [0,1] (countable covers)",
        "Ramsey's theorem (pairs) (illustrative)",
        "Nonexistent theorem example"
    ]
    for q in queries:
        print("Theorem:", q)
        w = db.weakest_subsystem_for(q)
        if w is None:
            print("  -> No subsystem in the current database proves this theorem (closure yields no proof).")
        else:
            print("  -> Weakest subsystem(s) (by rank) that prove it in the current DB:", ", ".join(w))
            path = db.explain_proof_path(q, w[0])
            if path:
                print("     Example derivation path in {}:".format(w[0]))
                for step in path:
                    print("       -", step)
        print("")
    closure = db.compute_closure_per_subsystem()
    for s in sorted(closure.keys(), key=lambda x: db.subsystems[x].rank):
        print("  {} (rank={}): {} theorems provable".format(s, db.subsystems[s].rank, len(closure[s])))


Theorem: Bolzano-Weierstrass theorem (every bounded sequence has a convergent subsequence)
  -> Weakest subsystem(s) (by rank) that prove it in the current DB: WKL0
     Example derivation path in WKL0:
       - Heine-Borel compactness for [0,1] (countable covers) (axiom/base in WKL0)
       - Existence of suprema for bounded sequences of rationals
       - Bolzano-Weierstrass theorem (every bounded sequence has a convergent subsequence)

Theorem: Heine-Borel compactness for [0,1] (countable covers)
  -> Weakest subsystem(s) (by rank) that prove it in the current DB: WKL0
     Example derivation path in WKL0:
       - Heine-Borel compactness for [0,1] (countable covers) (axiom/base in WKL0)

Theorem: Ramsey's theorem (pairs) (illustrative)
  -> Weakest subsystem(s) (by rank) that prove it in the current DB: ACA0
     Example derivation path in ACA0:
       - Some instances of Ramsey-like combinatorial principles (illustrative) (axiom/base in ACA0)
       - Ramsey's theorem (pairs) (ill